In [ ]:
import pandas as pd
import numpy as np

df=pd.read_csv('/kaggle/input/nlp-project/Sentiment_Stock_data.csv')


In [ ]:
import numpy as np

df1, df2, df3 = np.array_split(df, 3)

In [ ]:
df3.shape

(36250, 3)

In [ ]:
df1.columns=df.columns.str.replace(" ","_")
df1=df.drop(columns=['Unnamed:_0'])

In [ ]:
import string
# Fill NaN values in the 'Sentence' column with empty strings
df1['Sentence'] = df1['Sentence'].fillna('')

# Define the function to remove punctuation
exclude = string.punctuation
def remove_punc(text):
    # text is now guaranteed to be a string because we filled NaN with empty strings.
    return text.translate(str.maketrans('', '', exclude))

# Apply the remove_punc function to the 'Sentence' column
df1['Sentence'] = df1['Sentence'].apply(remove_punc)


In [ ]:
chat_words = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek You",
    "ILU": "I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A** Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A**",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A** Off",
    "SK8": "Skate",
    "STATS": "Your Sex and Age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F***",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laughter",
    "TFW": "That Feeling When",
    "MFW": "My Face When",
    "MRW": "My Reaction When",
    "IFYP": "I Feel Your Pain",
    "LOL": "Laughing Out Loud",
    "TNTL": "Trying Not To Laugh",
    "JK": "Just Kidding",
    "IDC": "I Don't Care",
    "ILY": "I Love You",
    "IMU": "I Miss You",
    "ADIH": "Another Day In Hell",
    "ZZZ": "Sleeping, Bored, Tired",
    "WYWH": "Wish You Were Here",
    "TIME": "Tears In My Eyes",
    "BAE": "Before Anyone Else",
    "FIMH": "Forever In My Heart",
    "BSAAW": "Big Smile And A Wink",
    "BWL": "Bursting With Laughter",
    "BFF": "Best Friends Forever",
    "CSL": "Can't Stop Laughing"
}

def chat_conversion(text):
  new_text=[]
  for w in text.split():
    if w.upper() in chat_words:
      new_text.append(chat_words[w.upper()])
    else:
      new_text.append(w)
  return ' '.join(new_text)

df1['Sentence']=df1['Sentence'].apply(chat_conversion)

In [ ]:
from textblob import TextBlob
df1['Sentence']=df1['Sentence'].apply(lambda x:str(TextBlob(x).correct()))

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
def remove_stopwords(text):
  new_text=[]
  for word in text.split():
    if word in stopwords.words('english'):
      new_text.append('')
    else:
      new_text.append(word)
  x=new_text[:]
  new_text.clear()
  return " ".join(x)

df1['Sentence']=df1['Sentence'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import spacy
nlp=spacy.load("en_core_web_sm")
df1['lemmas']=df1['Sentence'].apply(lambda x: " ".join([token.lemma_ for token in nlp(x)]))

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
import pandas as pd
import spacy
import re
from spacy.pipeline import EntityRuler
from spacy.tokenizer import Tokenizer

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

# Custom tokenizer to split "EUR123" into "EUR" and "123"
def custom_tokenizer(nlp):
    infix_re = re.compile(r"(?<=[A-Za-z])(?=\d)|(?<=\d)(?=[A-Za-z])")
    return Tokenizer(nlp.vocab, infix_finditer=infix_re.finditer)

nlp.tokenizer = custom_tokenizer(nlp)

# Add EntityRuler for recognizing "EUR" followed by numbers as MONEY
ruler = nlp.add_pipe("entity_ruler", before="ner")
patterns = [{"label": "MONEY", "pattern": [{"LOWER": "eur"}, {"IS_DIGIT": True}]}]
ruler.add_patterns(patterns)


# Function to extract entities
def extract_entities(text):
    doc = nlp(str(text))  # Convert to string to avoid errors

    orgs = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    products = [ent.text for ent in doc.ents if ent.label_ == "PRODUCT"]
    money = [ent.text for ent in doc.ents if ent.label_ == "MONEY"]

    return {"ORG": ", ".join(orgs), "PRODUCT": ", ".join(products), "MONEY": ", ".join(money)}

# Apply entity extraction
df1["Entities"] = df1["Sentence"].apply(lambda x: extract_entities(x))

df_filtered = df_entities[df_entities["ORG"] != ""]

# Display the filtered DataFrame


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
from transformers import pipeline
import pandas as pd

# Load sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

# Function to analyze sentiment of the context
def analyze_sentiment(text):
    if not text.strip():  # Skip empty or whitespace-only texts
        return None, None
    result = sentiment_pipeline(text)[0]
    return result["label"].lower(), result["score"]  # Ensure lowercase labels

df1["lemmas"] = df1["lemmas"].fillna("")  # Replace NaNs with empty strings
df1[["Sentiments", "Confidence"]] = df1["lemmas"].apply(lambda x: pd.Series(analyze_sentiment(x)))

# Map sentiment labels to numeric values
df1["Sentiments"] = df1["Sentiments"].map({"positive": 1, "neutral": 0, "negative": -1})

df1 = df1.dropna(subset=["Sentiments", "Confidence"])

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [ ]:
df1['Sentiments'].unique()

array([0, 1])

In [ ]:
import pandas as pd
import spacy
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Load Sentiment Analysis Model
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

# Prepare ML Data
X = df1[["Sentiments", "Confidence"]]
y = df1["Sentiment"]

# Train Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict Stock Categories
df1["Predicted_Category"] = model.predict(X)



Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [ ]:
df1["Predicted_Category"].unique()

array([1, 0])

In [ ]:
df1["ORG"] = df1["Entities"].apply(lambda x: x.get("ORG", None))
df1["PRODUCT"] = df1["Entities"].apply(lambda x: x.get("PRODUCT", None))
df1["MONEY"] = df1["Entities"].apply(lambda x: x.get("MONEY", None))
df1.head(10)

KeyError: 'Entities'

In [ ]:
# Function to get dependency parsing for each word in a column
def dependency_parse(text):
    if pd.isna(text):  # Handle None or NaN values
        return None
    doc = nlp(str(text))
    return [(token.text, token.dep_, token.head.text) for token in doc]

# Apply dependency parsing to each column
print(df1["ORG"].apply(dependency_parse))
print(df1["PRODUCT"].apply(dependency_parse))
print(df1["MONEY"].apply(dependency_parse))

In [ ]:
import spacy
import pandas as pd
from spacy import displacy

# Function to display dependency tree
def visualize_dependency(text):
    if pd.isna(text):  # Handle None or NaN values
        return None
    doc = nlp(str(text))
    displacy.render(doc, style="dep", jupyter=True)  # Show visualization

# Example: Show dependency tree for "ORG" column (first row with non-null value)
for text in df1["ORG"].dropna():
    visualize_dependency(text)

In [ ]:
xx = df1.groupby("Predicted_Category")[["ORG", "PRODUCT", "MONEY"]].agg(list).reset_index()
xx

KeyError: 'Predicted_Category'

In [ ]:
xx.to_csv("df1.csv", index=False)  # Saves without the index column